In [1]:
import sys
import json
import torch
from pathlib import Path
import tensorboardX

# import locals
sys.path.insert(0, str(Path('..').resolve()))
from tlaloc import StockDataModule, SeqDataset
from tlaloc import StockGRUModel


In [2]:
trnsf_path = Path('../output/models/transforms.json').resolve()
model_path = Path('../output/models/model.pth').resolve()

In [3]:
with open(trnsf_path, 'r') as f:
    xform = json.load(f)
print(json.dumps(xform, indent=4))

{
    "min": 0.09027799963951111,
    "max": 76.41999816894531
}


In [4]:
model = StockGRUModel(hidden_dim=64, num_layers=4)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [5]:
msft = StockDataModule.get_stock_data('../data', 'MSFT')